# Preprocessing of Dataset
### 1. Motivation and Goal
This notebook is for preprocessing and cleaning the dataset. We are going to read the data a structure it in a coherent way. With cleaning we mean that all the columns in the dataset needs the correct type, we want numbers to be numbers (and not strings) and dates to be dates etc. We will also find a way to deal with missing values and drop or transform columns which are not needed / in need of a transformation. 

The output of this notebook is a csv-file is more suitable to do exploratory data analysis on compared to the raw data. 

### 2. Imports

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pandas_ods_reader import read_ods

### 3. Read Data

In [130]:
class ReadData:
    def __init__(self, file_path):
        """
        The .ods-file consists of sheets which all are read individualy.
        This class reads all the sheets and concatenates them into a single
        pandas DataFrame.
        """
        sheet = 0
        self.df = read_ods(file_path, sheet)
        while True:
            try:
                new_sheet = read_ods(file_path, sheet)
                new_sheet = new_sheet.drop(0)
                self.df = pd.concat([self.df, new_sheet])
                sheet += 1
            except IndexError as e:
                break
        self.descriptions = self.df.iloc[0].to_dict()
        self.df = self.df.drop(0) 
        
    def get_data(self):
        """
        Retrieve the DataFrame and a dictionary with descriptions
        of the columns in the DataFrame
        """
        return self.df, self.descriptions

In [131]:
# Read the data
file_path = "./data/glofdatabase_V3.ods"
rd = ReadData(file_path)
original_df, COLUMN_DESCRIPTIONS = rd.get_data()

In [132]:
# View head of df
original_df.head()

,ID,Major_RGI_Region,Mountain_range_Region,Country,Glacier,RGI_Glacier_Id,RGI_Glacier_Area,Lake,Lake_type,Longitude,Latitude,River,Date,Date_Min,Date_Max,Mechanism,Mean_Lake_Volume_VL,Min_VL,Max_VL,VL_calculation,Mean_Flood_Volume_V0,Min_V0,Max_V0,V0_calculation,Peak_discharge_Qp,Min_Qp,Max_Qp,Qp_calculation,Impact_and_destruction,reported_impacts,economic_losses,D_buildings,D_bridges,D_roads_paths,D_railroads,D_utilities,D_flood_protection,D_environmental,resettlement,reported_fatalities,Further_comments,First_reference_found,Reference,Reference_comments,Lake_area_before,Perimeter_before,Image_date_before,Satellite_before,Scenename_before,Certainty_level_before,Lake_area_after,Perimeter_after,Image_date_after,Satellite_after,Scenename_after,Certainty_level_after,Lake_area_comments,unnamed.1,unnamed.3,unnamed.4
1,None,None,e.g. Karakoram,e.g. Pakistan,e.g. Baltoro Glacier,None,km2,e.g. Baltoro Lake 1,"e.g ice, moraine, water pocket, bedrock, combined",XX.XX°,XX.XX°,e.g. Indus,YYYY-MM-DD,YYYY-MM-DD,YYYY-MM-DD,"(e.g. overtopping, breach, tunnelling, englac...",106 m³,106 m³,106 m³,e.g. bathymetry,106 m³,106 m³,106 m³,gaged / estmated / unknown,m³ s-1,m³ s-1,m³ s-1,gaged / estmated / unknown,None,None,None,None,None,None,None,(e.g. water or electricity supply),(e.g. levees),(e.g. fish population),None,None,None,None,None,None,m²,m,YYYY-MM_DD,None,None,2 – sure,m²,m,YYYY-MM_DD,None,None,2 – sure,None,NaN,NaN,NaN
2,1.0,None,None,Greenland,None,RGI60-05.00334,238.201,Iluliallup Tasersua,ice,-51.815633,65.77412,None,1936.0,1948.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,lake known to drain every five to seven years;...,1966.0,"Helk, C. J. (1966). Glacier mapping in Greenla...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN
3,2.0,None,None,Greenland,Sermilik Bræ,None,None,Imaersartoq,ice,-48.454648,61.954559,None,1942.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"drains annualy, usually in early august (proba...",1970.0,"Higgins, A. K. (1970). On some ice-dammed lake...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN
4,3.0,None,None,Greenland,Sermiligårssuk Bræ,None,None,North Midternæs temporary lake,ice,-47.965525,61.676943,None,1942.0,None,1942-07-24,tunnelling,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,approximate volume of water released at times ...,1970.0,"Higgins, A. K. (1970). On some ice-dammed lake...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN
5,4.0,None,None,Greenland,Sermilik Bræ,None,None,Imaersartoq,ice,-48.454648,61.954559,None,1943.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"drains annualy, usually in early august (proba...",1970.0,"Higgins, A. K. (1970). On some ice-dammed lake...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN


In [133]:
# View descriptions of some columns
print(COLUMN_DESCRIPTIONS['Country'])
print(COLUMN_DESCRIPTIONS['ID'])

Source location of the GLOF
running number (regionally)


### 4. Preprocessing and Cleaning

In [134]:
# print number of columns and rows
print("Number of rows in dataset:", len(original_df))
print("Number of columns in dataset", len(original_df.columns))

Number of rows in dataset: 3467
Number of columns in dataset 60


In [135]:
# See the types of the columns
original_df.dtypes

ID                         object
Major_RGI_Region           object
Mountain_range_Region      object
Country                    object
Glacier                    object
RGI_Glacier_Id             object
RGI_Glacier_Area           object
Lake                       object
Lake_type                  object
Longitude                  object
Latitude                   object
River                      object
Date                       object
Date_Min                   object
Date_Max                   object
Mechanism                  object
Mean_Lake_Volume_VL        object
Min_VL                     object
Max_VL                     object
VL_calculation             object
Mean_Flood_Volume_V0       object
Min_V0                     object
Max_V0                     object
V0_calculation             object
Peak_discharge_Qp          object
Min_Qp                     object
Max_Qp                     object
Qp_calculation             object
Impact_and_destruction     object
reported_impac

**Comment:** Every column is a string apparently.

In [136]:
COLUMN_DESCRIPTIONS['economic_losses']

'Occurrence of economic losses'

In [137]:
original_df['unnamed.4'].unique()

array([      nan, 12.601757])

In [138]:
original_df.sample(1)

,ID,Major_RGI_Region,Mountain_range_Region,Country,Glacier,RGI_Glacier_Id,RGI_Glacier_Area,Lake,Lake_type,Longitude,Latitude,River,Date,Date_Min,Date_Max,Mechanism,Mean_Lake_Volume_VL,Min_VL,Max_VL,VL_calculation,Mean_Flood_Volume_V0,Min_V0,Max_V0,V0_calculation,Peak_discharge_Qp,Min_Qp,Max_Qp,Qp_calculation,Impact_and_destruction,reported_impacts,economic_losses,D_buildings,D_bridges,D_roads_paths,D_railroads,D_utilities,D_flood_protection,D_environmental,resettlement,reported_fatalities,Further_comments,First_reference_found,Reference,Reference_comments,Lake_area_before,Perimeter_before,Image_date_before,Satellite_before,Scenename_before,Certainty_level_before,Lake_area_after,Perimeter_after,Image_date_after,Satellite_after,Scenename_after,Certainty_level_after,Lake_area_comments,unnamed.1,unnamed.3,unnamed.4
203,202.0,Alaska,W Chugach Mtns (Talkeetna),USA,Snow,RGI60-01.09061,50.928,Snow,ice,-148.924999,60.482579,None,1951-11-06,None,None,None,None,None,None,None,97.074876,None,None,estimated,311.48,None,None,estimated peak outflow of the lake,None,None,None,None,None,None,None,None,None,None,None,None,None,1971.0,"Post, A., & Mayo, L. R. (1971). Glacier dammed...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN


In [139]:
# Find numerical columns and date columns
numerical_columns = ['ID', 'RGI_Glacier_Area', 'Longitude', 'Latitude', 'Mean_Lake_Volume_VL', 'Min_VL', 'Max_VL', 'Mean_Flood_Volume_V0', 'Min_V0',
'Max_V0', 'Peak_discharge_Qp', 'Min_Qp', 'Max_Qp', 'D_buildings', 'D_bridges', 'D_roads_paths', 'reported_fatalities', 'First_reference_found',
'Lake_area_before', 'Perimeter_before', 'Certainty_level_before', 'Lake_area_after', 'Perimeter_after', 'Certainty_level_after']
date_columns = ['Date', 'Date_Min', 'Date_Max', 'Image_date_before', 'Image_date_after']

In [143]:
import datetime as dt

def convert_to_float(x):
    try:
        x = float(x)
    except Exception as e:
        x = np.NaN
    return x

def convert_to_datetime(x):
    try:
        x = x.split('-')
        if len(x) == 1:
            x = dt.datetime(int(x[0]), 1, 1)
            return x
        elif len(x) == 2:
            x = dt.datetime(int(x[0], int(x[1]), 1))
            return x
        elif len(x) == 3:
            x = dt.datetime(int(x[0]), int(x[1]), int(x[2]))
            return x
    except AttributeError as e:
        x = dt.datetime(x, 1, 1)
        return x

def convert_columns_to_numerical(columns, df: pd.DataFrame):
    for col in columns:
        df[col] = df[col].apply(convert_to_float)

    return df

def convert_columns_to_date(columns, df: pd.DataFrame):
    for col in columns:
        df[col] = df[col].apply(convert_to_datetime)

    return df

In [144]:
df = original_df.copy()
df = convert_columns_to_numerical(numerical_columns, df)
df = convert_columns_to_date(date_columns, df)

ValueError: invalid literal for int() with base 10: 'YYYY'

In [95]:
original_df['RGI_Glacier_Area']

1       -1.000
2      238.201
3       -1.000
4       -1.000
5       -1.000
        ...   
150     35.228
151     -1.000
152     -1.000
153     -1.000
154     -1.000
Name: RGI_Glacier_Area, Length: 3467, dtype: float64